In [1]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tld

     |████████████████████████████████| 412 kB 5.0 MB/s 


In [3]:
path = '/content/drive/MyDrive/verified_online.csv'

In [4]:
df = pd.read_csv(path)   # 데이터 로드
df.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7234051,http://seguridadar7912.byethost9.com/pase.php,http://www.phishtank.com/phish_detail.php?phis...,2021-07-19T04:06:30+00:00,yes,2021-07-19T04:15:44+00:00,yes,Other
1,7234046,http://kpatacusot.com/first/mazon/amazon/homep...,http://www.phishtank.com/phish_detail.php?phis...,2021-07-19T04:00:33+00:00,yes,2021-07-19T04:06:20+00:00,yes,Other
2,7234045,http://kpatacusot.com/first/mazon/f7404/,http://www.phishtank.com/phish_detail.php?phis...,2021-07-19T04:00:30+00:00,yes,2021-07-19T04:06:20+00:00,yes,Other
3,7234044,https://hufschuh.app/online.americanexpress.com,http://www.phishtank.com/phish_detail.php?phis...,2021-07-19T04:00:25+00:00,yes,2021-07-19T04:06:20+00:00,yes,Other
4,7234043,http://kpatacusot.com/first/mazon/amazon/,http://www.phishtank.com/phish_detail.php?phis...,2021-07-19T04:00:20+00:00,yes,2021-07-19T04:06:20+00:00,yes,Other


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10639 entries, 0 to 10638
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   phish_id           10639 non-null  int64 
 1   url                10639 non-null  object
 2   phish_detail_url   10639 non-null  object
 3   submission_time    10639 non-null  object
 4   verified           10639 non-null  object
 5   verification_time  10639 non-null  object
 6   online             10639 non-null  object
 7   target             10639 non-null  object
dtypes: int64(1), object(7)
memory usage: 665.1+ KB


## 데이터 전처리

In [6]:
# url 단축 여부
import requests

# 시간당 100 request 한정
def url_shorten(url):
    req = 'http://expandurl.com/api/v1/?url='+url+'&format=json&detailed=true'
    response = requests.request("GET", req).json()
    if response['url'] != url:    # 단축 url인 경우 -> 피싱
        return 1
    else:                         # 정상
        return -1

In [21]:
# url 단축 여부
import requests

# 만족하는 경우 별로 없음
def url_shorten_2(url):
    req = 'https://unshorten.me/json/'+url
    response = requests.request("GET", req).json()
    if (response['success'] and response['resolved_url'] != url):    # 단축 url인 경우 -> 피싱
        return 1
    else:                                                            # 정상
        return -1

In [22]:
# url 길이

def length(url):
    return len(url)

In [23]:
# @ 확인

def sigh(url):
    if '@' in url:  # 피싱
        return 1
    else:           # 피싱
        return -1

In [24]:
# redirection 확인
import urllib.parse

def redirection(url):
    parse = urllib.parse.urlparse(url)
    path = parse.path

    if '//' in path: # 피싱
        return 1
    else:            # 정상
        return -1

In [25]:
# IP 포함 확인

def IPAddress(url) :
    parse = urllib.parse.urlparse(url)
    domain = parse.netloc.split('.')

    try :
        int(domain[-1], 0)      # 피싱
        return 1
    except ValueError :         # 정상
        return -1

In [26]:
# http로 시작하는지 확인

def http(url):
    if url.startswith('http://'):   # 피싱
        return 1
    else:                           # 정상
        return -1

In [27]:
# - 확인

def dash(url):
    if '-' in url:  # 피싱
        return 1
    else:           # 정상
        return -1

In [28]:
# 사이트, 브랜드명 사용 여부
from bs4 import BeautifulSoup

topsite = []
response = requests.get('https://www.alexa.com/topsites')
html = response.text
soup = BeautifulSoup(html, 'html.parser')
for content in soup.select('#alx-content > div.row-fluid.TopSites.AlexaTool.padding20 > section.page-product-content.summary.padding20 > span > span > div > div > div.listings.table > div.tr.site-listing > div.td.DescriptionCell > p > a'):
    topsite.append(content.get_text().split('.')[0].lower())

def topsites(url):
    domain = urllib.parse.urlparse(url).netloc.lower()
    for site in topsite:
        if site in domain:      # 피싱
            return 1
    return -1                   # 정상

In [29]:
# . 개수

def dot(url):
    return url.count('.')

In [31]:
df_list = []
index = 0

for url in df['url'][:100]:
    df_list.append([])

    df_list[index].append(url_shorten_2(url))
    df_list[index].append(length(url))
    df_list[index].append(sigh(url))
    df_list[index].append(redirection(url))
    df_list[index].append(IPAddress(url))
    df_list[index].append(http(url))
    df_list[index].append(dash(url))
    df_list[index].append(topsites(url))
    df_list[index].append(dot(url))

    index += 1

In [32]:
for i in range(10):
    print(df_list[i])

[-1, 45, -1, -1, -1, 1, -1, -1, 3]
[-1, 50, -1, -1, -1, 1, -1, -1, 1]
[-1, 40, -1, -1, -1, 1, -1, -1, 1]
[-1, 47, -1, -1, -1, -1, -1, -1, 3]
[-1, 41, -1, -1, -1, 1, -1, -1, 1]
[-1, 37, -1, -1, -1, -1, -1, -1, 2]
[-1, 36, -1, -1, -1, 1, -1, -1, 2]
[-1, 36, -1, -1, -1, -1, -1, -1, 2]
[-1, 34, -1, -1, -1, -1, -1, -1, 2]
[-1, 34, -1, -1, -1, -1, -1, -1, 2]


## 데이터프레임 변환

In [33]:
columns = ['url_shorten', 'length', 'sigh', 'redirection', 'IP', 'http', 'dash', 'topsites', 'dot']
preprocessed_df = pd.DataFrame(df_list, columns=columns)

In [34]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   url_shorten  100 non-null    int64
 1   length       100 non-null    int64
 2   sigh         100 non-null    int64
 3   redirection  100 non-null    int64
 4   IP           100 non-null    int64
 5   http         100 non-null    int64
 6   dash         100 non-null    int64
 7   topsites     100 non-null    int64
 8   dot          100 non-null    int64
dtypes: int64(9)
memory usage: 7.2 KB


In [35]:
preprocessed_df.head()

,url_shorten,length,sigh,redirection,IP,http,dash,topsites,dot
0,-1,45,-1,-1,-1,1,-1,-1,3
1,-1,50,-1,-1,-1,1,-1,-1,1
2,-1,40,-1,-1,-1,1,-1,-1,1
3,-1,47,-1,-1,-1,-1,-1,-1,3
4,-1,41,-1,-1,-1,1,-1,-1,1


In [36]:
preprocessed_df.describe()

,url_shorten,length,sigh,redirection,IP,http,dash,topsites,dot
count,100.000000,100.000000,100.00,100.0,100.0,100.000000,100.000000,100.000000,100.00000
mean,-0.680000,59.860000,-0.98,-1.0,-1.0,-0.080000,-0.240000,-0.940000,2.33000
std,0.736906,50.509529,0.20,0.0,0.0,1.001817,0.975663,0.342893,1.35628
min,-1.000000,19.000000,-1.00,-1.0,-1.0,-1.000000,-1.000000,-1.000000,1.00000
25%,-1.000000,36.000000,-1.00,-1.0,-1.0,-1.000000,-1.000000,-1.000000,2.00000
50%,-1.000000,42.500000,-1.00,-1.0,-1.0,-1.000000,-1.000000,-1.000000,2.00000
75%,-1.000000,67.750000,-1.00,-1.0,-1.0,1.000000,1.000000,-1.000000,2.25000
max,1.000000,373.000000,1.00,-1.0,-1.0,1.000000,1.000000,1.000000,12.00000
